In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [105]:
def cnn_model(features, labels, mode):
    input_shape = tf.reshape(features["x"],[-1,28,28,1])
    conv1 = tf.layers.conv2d(inputs=input_shape,kernel_size=(5,5),filters=32,padding="same",activation='elu')
    pooling1 = tf.layers.max_pooling2d(conv1,pool_size=(2,2),strides=2)
    conv2 = tf.layers.conv2d(inputs=pooling1,kernel_size=5,filters=64,padding="same",activation='elu')
    pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=2,strides=2)
    pool2_flat = tf.reshape(pool2,[-1,7*7*64])
    dense = tf.layers.dense(units=1024,activation='elu',inputs=pool2_flat)
    dropout = tf.layers.dropout(rate = 0.1,inputs=dense,training=mode==tf.estimator.ModeKeys.TRAIN)
    dense2 = tf.layers.dense(units=10,inputs=dropout)
    softmax = tf.nn.softmax(logits=dense2,name="softmax_tensor")
    predictions={
        "classes":tf.argmax(input=dense2,axis=1),
        "probablities":tf.nn.softmax(dense2,name='softmax_tensor')
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=dense2)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,train_op=train_op,loss=loss)
    
    eval_metrics = {
        "accuracy":tf.metrics.accuracy(labels=labels,predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metrics)



In [106]:
((train_data,train_labels),(eval_data,eval_labels))=tf.keras.datasets.mnist.load_data()
train_data = train_data/np.float32(255)
eval_data = eval_data/np.float32(255)
train_labels = train_labels.astype(np.int32)
eval_labels = eval_labels.astype(np.int32)

In [107]:
mnist_classifer = tf.estimator.Estimator(model_fn=cnn_model,model_dir='/tmp/mnist_convnet_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x162b375c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [108]:
tensors_to_logs={"probablitities":"softmax_tensor"}

logging_hook=tf.train.LoggingTensorHook(tensors=tensors_to_logs,every_n_iter=50)

In [109]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":train_data},y=train_labels,batch_size=100,num_epochs=None,shuffle=True)

In [110]:
mnist_classifer.train(train_input_fn,hooks=[logging_hook],steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 200 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probablitities = [[0.00330404 0.00002207 0.00003102 0.01157629 0.00012918 0.78045374
  0.0000129  0.18522495 0.01760362 0.00164214]
 [0.00013517 0.0000222  0.00001502 0.00027606 0.00079857 0.00033401
  0.00000889 0.9894014  0.00137321 0.00763537]
 [0.00007378 0.00000204 0.00000214 0.00011354 0.00003376 0.00032058
  0.00000091 0.99699473 0.00100994 0.00144863]
 [0.00180994 0.00003361 0.00350131 0.0005032  0.14480099 0.00153924
  0.8070072  0.00081946 0.02666184 0.01332322]
 [0.03689324 0.00035108 0.00038305 0.00083468 0.10809895 0.5339608
  0.00234113 0.19754784 0.06518941 0

INFO:tensorflow:loss = 0.54884887, step = 201
INFO:tensorflow:probablitities = [[0.00002577 0.9851248  0.00262848 0.00181362 0.00041134 0.00013276
  0.00180349 0.0007991  0.00646772 0.00079278]
 [0.00655798 0.00001059 0.06017284 0.00121258 0.00086534 0.003155
  0.89932394 0.00000877 0.02857779 0.00011515]
 [0.5710453  0.00014912 0.33932588 0.00273735 0.00028433 0.03246186
  0.04475467 0.00084769 0.00738016 0.00101362]
 [0.00085446 0.00129099 0.01139476 0.01247833 0.00609469 0.00585971
  0.00443593 0.00693609 0.92025524 0.03039983]
 [0.02852821 0.08531624 0.00689552 0.03012716 0.06231023 0.5008869
  0.00684168 0.03352364 0.11951545 0.12605496]
 [0.00013913 0.9625202  0.00467633 0.00442936 0.00134329 0.00232941
  0.00865854 0.00208277 0.01078276 0.00303827]
 [0.00065826 0.00000458 0.00003224 0.00410687 0.0155972  0.00289061
  0.00015199 0.13111614 0.00217046 0.8432716 ]
 [0.00026173 0.00009215 0.00336238 0.9859861  0.00000068 0.00628161
  0.00001131 0.00016763 0.00374865 0.00008777]
 [0.

INFO:tensorflow:Saving checkpoints for 300 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.4277894.


In [111]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":eval_data},y=eval_labels,shuffle=True,num_epochs=3)

In [113]:
eval_res = mnist_classifer.evaluate(input_fn=eval_input_fn,steps=10)
print(eval_res)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-03T09:21:06Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2019-05-03-09:21:08
INFO:tensorflow:Saving dict for global step 300: accuracy = 0.9351562, global_step = 300, loss = 0.28099504
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 300: /tmp/mnist_convnet_model/model.ckpt-300
{'accuracy': 0.9351562, 'loss': 0.28099504, 'gl